In [18]:
from hyptorch.geoopt import PoincareBall   
from hyptorch.lorentz.manifold import CustomLorentz 
from model.modules.utils import ManifoldMapper
import torch


poincare = PoincareBall(c=1.0, learnable=True)
lorentz = CustomLorentz(k=1.0, learnable=True)

lmapper = ManifoldMapper(lorentz, clip_r=1.25, curv=lorentz.k)
pmapper = ManifoldMapper(lorentz, clip_r=1.25, curv=poincare.c)
vis_embed = torch.rand(100, 12, 256)
text_embed = torch.rand(100, 256)
print(vis_embed.T.shape)


vis_embed = lmapper(vis_embed)
text_embed = lmapper(text_embed)
lorentz.assert_check_point_on_manifold(vis_embed)
print('text_embed', text_embed.shape)
print('vision_embed', vis_embed.shape)

sim_q2t,_ = lorentz.dist_batch(
    vis_embed.unsqueeze(1), 
    text_embed.unsqueeze(-1),
).squeeze().max(1)


print(sim_q2t)
sim_t2q,_ = lorentz.dist_batch(
    text_embed.unsqueeze(1).unsqueeze(1),
    vis_embed.permute(0, 2, 1),
).squeeze().max(1)
print(sim_t2q)


torch.Size([256, 12, 100])
text_embed torch.Size([100, 257])
vision_embed torch.Size([100, 12, 257])
tensor([[1.1687, 1.1688, 1.1689,  ..., 1.1458, 1.1415, 1.1293],
        [1.1008, 1.1727, 1.1338,  ..., 1.1007, 1.1767, 1.1207],
        [1.1291, 1.1587, 1.1833,  ..., 1.1119, 1.1322, 1.1551],
        ...,
        [1.1232, 1.1703, 1.1322,  ..., 1.1180, 1.1537, 1.1263],
        [1.1470, 1.1715, 1.1758,  ..., 1.1159, 1.1572, 1.1315],
        [1.1334, 1.1306, 1.1603,  ..., 1.1238, 1.1147, 1.1224]],
       grad_fn=<MaxBackward0>)
tensor([[1.1687, 1.1008, 1.1291,  ..., 1.1232, 1.1470, 1.1334],
        [1.1688, 1.1727, 1.1587,  ..., 1.1703, 1.1715, 1.1306],
        [1.1689, 1.1338, 1.1833,  ..., 1.1322, 1.1758, 1.1603],
        ...,
        [1.1458, 1.1007, 1.1119,  ..., 1.1180, 1.1159, 1.1238],
        [1.1415, 1.1767, 1.1322,  ..., 1.1537, 1.1572, 1.1147],
        [1.1293, 1.1207, 1.1551,  ..., 1.1263, 1.1315, 1.1224]],
       grad_fn=<MaxBackward0>)


In [35]:
import torch.nn.functional as F
def itc_loss(self, image_embeds, text_embeds, image_worlds, text_worlds, sim_i2t_targets, sim_t2i_targets):
    sim_i2t = self.dist_func(image_embeds, text_worlds) 
    sim_t2i = self.dist_func(text_embeds, image_worlds)

    loss_i2t = -torch.sum(
        F.log_softmax(sim_i2t / self.logit_scale, dim=1) * sim_i2t_targets, dim=-1
    ).mean()
    loss_t2i = -torch.sum(
        F.log_softmax(sim_t2i / self.logit_scale, dim=1) * sim_t2i_targets, dim=-1
    ).mean()      

    loss_itc = self.config.weight_i2t * loss_i2t + (1-self.config.weight_i2t) * loss_t2i
    return loss_itc

